# Projeto Final

## 1. Carregando os dados

In [96]:
import pandas as pd
import numpy as np

import warnings

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

from lightgbm import LGBMClassifier

In [97]:
warnings.filterwarnings(
    "ignore",
    message="X does not have valid feature names, but LGBMClassifier was fitted with feature names"
)

In [98]:
df_contract = pd.read_csv('datasets/contract.csv')

In [99]:
df_personal = pd.read_csv('datasets/personal.csv')

In [100]:
df_internet = pd.read_csv('datasets/internet.csv')

In [101]:
df_phone = pd.read_csv('datasets/phone.csv')

In [102]:
df_contract.info()
df_contract.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65


In [103]:
df_personal.info()
df_personal.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


In [104]:
df_internet.info()
df_internet.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


In [105]:
df_phone.info()
df_phone.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


Analisando os dados, pude perceber que na tabela `contract` há alguns tipos de dados a ser corrigidos. Fora isso, parece que o resto dos dados está em ordem.

## 2. AED

### 2.1. Corrigindo tipos de dados

#### Tabela `contract`

In [106]:
df_contract['BeginDate'] = pd.to_datetime(df_contract['BeginDate'], errors='coerce')

In [107]:
df_contract['EndDate'] = pd.to_datetime(df_contract['EndDate'], errors='coerce')

C:\Users\joaod\AppData\Local\Temp\ipykernel_35852\2334116791.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_contract['EndDate'] = pd.to_datetime(df_contract['EndDate'], errors='coerce')


In [108]:
df_contract['TotalCharges'] = (
    df_contract['TotalCharges']
    .replace(' ', np.nan)
    .astype(float)
)

In [109]:
df_contract.info()
df_contract.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerID        7043 non-null   object        
 1   BeginDate         7043 non-null   datetime64[ns]
 2   EndDate           1869 non-null   datetime64[ns]
 3   Type              7043 non-null   object        
 4   PaperlessBilling  7043 non-null   object        
 5   PaymentMethod     7043 non-null   object        
 6   MonthlyCharges    7043 non-null   float64       
 7   TotalCharges      7032 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(4)
memory usage: 440.3+ KB


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50
2,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,NaT,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65


### 2.2. Engenharia de atributos

#### Criando o objetivo

In [110]:
df_contract['churn'] = df_contract['EndDate'].notna().astype(int)

#### Novas colunas úteis para o modelo

In [111]:
reference_date = pd.to_datetime('2020-02-01')

Dias e meses da duração do contrato:

In [112]:
df_contract['tenure_days'] = (
    df_contract['EndDate']
    .fillna(reference_date) 
    - df_contract['BeginDate']
).dt.days

df_contract['tenure_months'] = df_contract['tenure_days'] / 30

Se é um novo cliente:

In [113]:
df_contract['is_new_costumer'] = (df_contract['tenure_months'] < 3).astype(int)

Gasto médio mensal:

In [114]:
df_contract['avg_monthly_spend'] = df_contract['TotalCharges'] / df_contract['tenure_months'].replace(0, np.nan)

Se gasta mais que a mediana:

In [115]:
df_contract['high_monthly_charges'] = (df_contract['MonthlyCharges'] > df_contract['MonthlyCharges'].median()).astype(int)

Se o tipo de pagamento é mensal:

In [116]:
df_contract['is_month_to_month'] = (df_contract['Type'] == 'Month-to-month').astype(int)

Se a forma de pagamento é 100% eletrônica:

In [117]:
df_contract['paperless_and_electronic'] = (
    (df_contract['PaperlessBilling'] == 'Yes') &
    (df_contract['PaymentMethod'] == 'Electronic check')
).astype(int)

Quantos serviços assina:

In [118]:
service_cols = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

df_internet['num_services'] = df_internet[service_cols].apply(
    lambda x: (x == 'Yes').sum(), axis=1
)

Se não assina nenhum serviço:

In [119]:
df_internet['has_no_extras'] = (df_internet['num_services'] == 0).astype(int)

In [120]:
df_contract.head()

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,churn,tenure_days,tenure_months,is_new_costumer,avg_monthly_spend,high_monthly_charges,is_month_to_month,paperless_and_electronic
0,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,0,31,1.033333,1,28.887097,0,1,1
1,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,0,1036,34.533333,0,54.715251,0,0,0
2,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,1,61,2.033333,1,53.188525,0,1,0
3,7795-CFOCW,2016-05-01,NaT,One year,No,Bank transfer (automatic),42.30,1840.75,0,1371,45.700000,0,40.278993,0,0,0
4,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,1,61,2.033333,1,74.581967,1,1,1


In [121]:
df_internet.head()

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,num_services,has_no_extras
0,7590-VHVEG,DSL,No,Yes,No,No,No,No,1,0
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No,2,0
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No,2,0
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No,3,0
4,9237-HQITU,Fiber optic,No,No,No,No,No,No,0,1


### 2.3. Criando X e y

#### Merge + removendo colunas inúteis para o modelo

In [122]:
df = df_contract \
    .merge(df_personal, on='customerID', how='left') \
    .merge(df_internet, on='customerID', how='left') \
    .merge(df_phone, on='customerID', how='left')

In [123]:
X = df.drop(columns=['customerID', 'BeginDate', 'EndDate', 'churn'])

In [124]:
X.info()
X.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Type                      7043 non-null   object 
 1   PaperlessBilling          7043 non-null   object 
 2   PaymentMethod             7043 non-null   object 
 3   MonthlyCharges            7043 non-null   float64
 4   TotalCharges              7032 non-null   float64
 5   tenure_days               7043 non-null   int64  
 6   tenure_months             7043 non-null   float64
 7   is_new_costumer           7043 non-null   int64  
 8   avg_monthly_spend         7032 non-null   float64
 9   high_monthly_charges      7043 non-null   int64  
 10  is_month_to_month         7043 non-null   int64  
 11  paperless_and_electronic  7043 non-null   int64  
 12  gender                    7043 non-null   object 
 13  SeniorCitizen             7043 non-null   int64  
 14  Partner 

,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,tenure_days,tenure_months,is_new_costumer,avg_monthly_spend,high_monthly_charges,...,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,num_services,has_no_extras,MultipleLines
0,Month-to-month,Yes,Electronic check,29.85,29.85,31,1.033333,1,28.887097,0,...,DSL,No,Yes,No,No,No,No,1.0,0.0,NaN
1,One year,No,Mailed check,56.95,1889.50,1036,34.533333,0,54.715251,0,...,DSL,Yes,No,Yes,No,No,No,2.0,0.0,No
2,Month-to-month,Yes,Mailed check,53.85,108.15,61,2.033333,1,53.188525,0,...,DSL,Yes,Yes,No,No,No,No,2.0,0.0,No
3,One year,No,Bank transfer (automatic),42.30,1840.75,1371,45.700000,0,40.278993,0,...,DSL,Yes,No,Yes,Yes,No,No,3.0,0.0,NaN
4,Month-to-month,Yes,Electronic check,70.70,151.65,61,2.033333,1,74.581967,1,...,Fiber optic,No,No,No,No,No,No,0.0,1.0,No


In [125]:
y = df['churn']

In [126]:
y.value_counts(normalize=True)

churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64

#### Separando treino e teste

In [127]:
X = X.replace([np.inf, -np.inf], np.nan)

In [128]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=22
)

#### Pré-processamento

In [129]:
numeric_features = X.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

In [130]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

Aqui precisei fazer alguns ajustes para lidar com NaNs que estavam atrapalhando os modelos.

## 3. Treinando os modelos

#### Função para avaliar modelos

In [131]:
def eval_model(pipe, X_test, y_test):
    y_proba = pipe.predict_proba(X_test)[:, 1]
    y_pred = pipe.predict(X_test)
    
    return {
        'ROC-AUC': roc_auc_score(y_test, y_proba),
        'F1': f1_score(y_test, y_pred),
        'Accuracy': accuracy_score(y_test, y_pred)
    }

#### Dummy

In [132]:
dummy_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', DummyClassifier(strategy='most_frequent', random_state=42))
])

dummy_pipe.fit(X_train, y_train)
eval_model(dummy_pipe, X_test, y_test)

{'ROC-AUC': np.float64(0.5), 'F1': 0.0, 'Accuracy': 0.7345635202271115}

#### Logistic Regression

In [133]:
lr_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        random_state=42
    ))
])

lr_pipe.fit(X_train, y_train)
eval_model(lr_pipe, X_test, y_test)

{'ROC-AUC': np.float64(0.8593621638378672),
 'F1': 0.644880174291939,
 'Accuracy': 0.7686302342086586}

#### Random Forest

In [134]:
rf_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(
        n_estimators=300,
        max_depth=12,
        class_weight='balanced',
        random_state=42
    ))
])

rf_pipe.fit(X_train, y_train)
eval_model(rf_pipe, X_test, y_test)

{'ROC-AUC': np.float64(0.8641155803559898),
 'F1': 0.656084656084656,
 'Accuracy': 0.815471965933286}

#### LightGBM

In [135]:
lgbm_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LGBMClassifier(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=6,
        class_weight='balanced',
        random_state=42,
        verbosity=-1
    ))
])

lgbm_pipe.fit(X_train, y_train)
eval_model(lgbm_pipe, X_test, y_test)

{'ROC-AUC': np.float64(0.9048347929422098),
 'F1': 0.7270440251572327,
 'Accuracy': 0.8459900638750887}

## 4. Avaliando os modelos

#### Validação cruzada

In [136]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [137]:
cv_auc_lr = cross_val_score(
    lr_pipe, X_train, y_train, cv=cv, scoring='roc_auc'
)

cv_f1_lr = cross_val_score(
    lr_pipe, X_train, y_train, cv=cv, scoring='f1'
)

cv_acc_lr = cross_val_score(
    lr_pipe, X_train, y_train, cv=cv, scoring='accuracy'
)

print(
    f"Logistic Regression | "
    f"ROC-AUC: {cv_auc_lr.mean():.3f} | "
    f"F1: {cv_f1_lr.mean():.3f} | "
    f"Accuracy: {cv_acc_lr.mean():.3f}"
)

Logistic Regression | ROC-AUC: 0.845 | F1: 0.620 | Accuracy: 0.742


In [138]:
cv_auc_rf = cross_val_score(
    rf_pipe, X_train, y_train, cv=cv, scoring='roc_auc'
)

cv_f1_rf = cross_val_score(
    rf_pipe, X_train, y_train, cv=cv, scoring='f1'
)

cv_acc_rf = cross_val_score(
    rf_pipe, X_train, y_train, cv=cv, scoring='accuracy'
)

print(
    f"Random Forest | "
    f"ROC-AUC: {cv_auc_rf.mean():.3f} | "
    f"F1: {cv_f1_rf.mean():.3f} | "
    f"Accuracy: {cv_acc_rf.mean():.3f}"
)

Random Forest | ROC-AUC: 0.859 | F1: 0.639 | Accuracy: 0.805


In [139]:
cv_auc_lgbm = cross_val_score(
    lgbm_pipe, X_train, y_train, cv=cv, scoring='roc_auc'
)

cv_f1_lgbm = cross_val_score(
    lgbm_pipe, X_train, y_train, cv=cv, scoring='f1'
)

cv_acc_lgbm = cross_val_score(
    lgbm_pipe, X_train, y_train, cv=cv, scoring='accuracy'
)

print(
    f"LightGBM | "
    f"ROC-AUC: {cv_auc_lgbm.mean():.3f} | "
    f"F1: {cv_f1_lgbm.mean():.3f} | "
    f"Accuracy: {cv_acc_lgbm.mean():.3f}"
)

LightGBM | ROC-AUC: 0.897 | F1: 0.703 | Accuracy: 0.829


#### Resultados sem validação cruzada

In [140]:
results = []

results.append({
    'model': 'Dummy',
    **eval_model(dummy_pipe, X_test, y_test)
})

results.append({
    'model': 'Logistic Regression',
    **eval_model(lr_pipe, X_test, y_test)
})

results.append({
    'model': 'RandomForest',
    **eval_model(rf_pipe, X_test, y_test)
})

results.append({
    'model': 'LightGBM',
    **eval_model(lgbm_pipe, X_test, y_test)
})

results = pd.DataFrame(results).sort_values(
    by='ROC-AUC',
    ascending=False
)

#### Resultados com a validação cruzada

In [141]:
results_cv = []

results_cv.append({
    'model': 'Logistic Regression',
    'ROC-AUC': cv_auc_lr.mean(),
    'F1': cv_f1_lr.mean(),
    'Accuracy': cv_acc_lr.mean()
})

results_cv.append({
    'model': 'Random Forest',
    'ROC-AUC': cv_auc_rf.mean(),
    'F1': cv_f1_rf.mean(),
    'Accuracy': cv_acc_rf.mean()
})

results_cv.append({
    'model': 'LightGBM',
    'ROC-AUC': cv_auc_lgbm.mean(),
    'F1': cv_f1_lgbm.mean(),
    'Accuracy': cv_acc_lgbm.mean()
})

results_cv = pd.DataFrame(results_cv).sort_values(
    by='ROC-AUC',
    ascending=False
)

#### Resultados finais

In [142]:
results

,model,ROC-AUC,F1,Accuracy
3,LightGBM,0.904835,0.727044,0.845990
2,RandomForest,0.864116,0.656085,0.815472
1,Logistic Regression,0.859362,0.644880,0.768630
0,Dummy,0.500000,0.000000,0.734564


In [143]:
results_cv

,model,ROC-AUC,F1,Accuracy
2,LightGBM,0.897226,0.703237,0.828544
1,Random Forest,0.859480,0.638584,0.805294
0,Logistic Regression,0.845226,0.620105,0.742104


## Conclusão

De todos os modelos treinados, o LightGBM se destacou nessa tarefa de classificação. Primeiramente comparei os resultados sem a validação cruzada, e o valor do AUC-ROC ficou em 0.9. Com o AUC-ROC de 0.89 na validação cruzada estratificada com cinco folds, isso prova que o modelo é confiável e sem overfitting.

Com base nos resultados obtidos, o modelo LightGBM foi selecionado como modelo final para a predição de churn de clientes da Interconnect.